In [6]:
# importing the libraries
import numpy as np
import pandas as pd
import matplotlib
import os
import cv2
import random
import keras

from glob import glob
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from keras.applications.mobilenet import MobileNet
import tensorflow as tf
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
# loading the directories 
training_dir = 'C:/Users/avadh/Desktop/CMPE 295A/Dataset/Try_Me/Train'
test_dir = 'C:/Users/avadh/Desktop/CMPE 295A/Dataset/Try_Me/Test'
folders=os.listdir("C:/Users/avadh/Desktop/CMPE 295A/Dataset/Try_Me/Train")
print(folders)
#test_dir = '../input/fruits/fruits-360_dataset/fruits-360/test-multiple_fruits/'

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/avadh/Desktop/CMPE 295A/Dataset/Try_Me/Train'

In [5]:
size=128,128

In [4]:
# useful for getting number of files
image_files = glob(training_dir + '/*/*.jp*g')
test_image_files = glob(test_dir + '/*/*.jp*g')


In [3]:
image_train=[]
image_labels=[]
image_names=[]

for folder in folders:
    if folder.endswith('jpg'):
        for each in os.listdir(os.path.join(training_dir,folder)):
            if each.endswith('jpg'):
                image_names.append(os.path.join(training_dir,folder,each))
                image_labels.append(folder)
                img=cv2.imread(os.path.join(training_dir,folder,each))
                img_in=cv2.resize(img,size)
                image_train.append(img_in)
            else:
                continue


NameError: name 'folders' is not defined

In [6]:
union_list=list(zip(image_train,image_labels))
random.shuffle(union_list)
train,labels=zip(*union_list)
X=np.array(train)
Y=np.array(labels)

In [7]:
X=X/255.0


In [8]:
le=LabelEncoder()
Y=le.fit_transform(Y)
Y=to_categorical(Y,num_classes=len(folders))

In [9]:
X_train,X_val,Y_train,Y_val=train_test_split(X,Y,test_size=0.1)


In [10]:
mobilenet_model=MobileNet(include_top=False,input_shape=(128,128,3))

In [11]:
model=Sequential()

for layer in mobilenet_model.layers:
    layer.trainable=False
for layers in mobilenet_model.layers:
    model.add(layers)
for layers in mobilenet_model.layers[-2:]:
    layers.trainable=True
    
model.add(Flatten())
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.5))

In [12]:
model.add(Dense(len(folders),activation='softmax'))


In [13]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1_pad (ZeroPadding2D)    (None, 129, 129, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 64, 64, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 64, 64, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 64, 64, 32)        128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 64, 64, 32)        0         
__________

In [14]:
optimizer=RMSprop(lr=0.0001)   #,rho=0.9,epsilon=1e-08,decay=0.0)
model.compile(optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
learning_rate_reduction=ReduceLROnPlateau(monitor='val_acc',patience=3,verbose=1,factor=0.5,minlr=0.00001)

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,
        rotation_range=10,  
        zoom_range = 0.1, 
        width_shift_range=0.1,
        height_shift_range=0.1,  
        horizontal_flip=False,
        vertical_flip=False) 

datagen.fit(X_train)

In [ ]:
history=model.fit_generator(datagen.flow(X_train,Y_train,batch_size=10),epochs=5,validation_data=(X_val,Y_val),
                          verbose=1,steps_per_epoch=X_train.shape[0]/10)


In [ ]:
from keras.models import model_from_json

# serialize model to JSON
model_json = model.to_json()
with open("Dataset/MyVersion/MobileNet.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")